In [1]:
pip install fastapi

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: exceptiongroup>=1.0.2 in c:\programdata\anaconda3\lib\site-packages (from anyio<5,>=3.6.2->starlette<0.47.0,>=0.40.0->fastapi) (1.2.2)

[notice] A new release of pip available: 22.1.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



In [2]:
pip install python-multipart


[notice] A new release of pip available: 22.1.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
from contextlib import asynccontextmanager
from fastapi import FastAPI, Request, Form
from fastapi.responses import HTMLResponse, RedirectResponse
from fastapi.templating import Jinja2Templates
from threading import Thread

In [4]:
from datetime import datetime
import json, time, random
import logging

In [9]:
# status_info 전역 변수 선언
status_info = {}

# 로그 설정
logging.basicConfig(
    filename="server.log",
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    encoding="utf-8"
)
log = logging.getLogger(__name__)

# FAST API 시작 부분 - app start 대체
@asynccontextmanager
async def lifespan(app: FastAPI):
    try:
        with open("tickers.json", "r") as f:
            tickers = json.load(f)
    except FileNotFoundError:
        tickers = []

    for entry in tickers:
        ex = entry["exchange"]
        if ex not in status_info:
            status_info[ex] = {
                "status": "Running",
                "last_updated": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            }
    # 앱 시작 시
    log.info("✅ App started")
    log.info("✅ Starting status_updater thread")
            
    thread = Thread(target=status_updater, daemon=True)
    thread.start()

    yield  # startup 이후 본격 앱 실행

    # 앱 종료
    log.info("🔴 App shutting down...")

# FastAPI 앱 선언 시 lifespan 추가
app = FastAPI(lifespan=lifespan)


In [10]:
templates = Jinja2Templates(directory="templates/") 

# 모든 html 파일은 templetes에 있음
# templates/index.html

def status_updater():
    """Background thread function to update exchange statuses and timestamps."""
    while True:
        for ex, info in list(status_info.items()):
            if info["status"] == "Running":
                # Update last-updated timestamp for running collectors
                info["last_updated"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            # Randomly simulate an error state (~10% chance each cycle)
            if random.random() < 0.1:
                info["status"] = "Error" if info["status"] == "Running" else "Running"
        time.sleep(5)  # wait 5 seconds before next update cycle

@app.get("/", response_class=HTMLResponse)
def show_dashboard(request: Request, q: str = None):
    """Dashboard page showing ticker list and status table. Supports optional search query."""
    # Read the latest tickers list from file
    try:
        with open("tickers.json", "r") as f:
            tickers = json.load(f)
    except FileNotFoundError:
        tickers = []
    # If a search query is provided, filter the tickers
    if q:
        query = q.lower()
        tickers = [t for t in tickers 
                   if query in t["exchange"].lower() or query in t["ticker"].lower()]
    # Sort tickers by exchange then ticker for display order
    tickers.sort(key=lambda x: (x["exchange"], x["ticker"]))
    # Render the HTML template with tickers and status information
    return templates.TemplateResponse("index.html", {
        "request": request,
        "tickers": tickers,
        "status_info": status_info,
        "query": q
    })

@app.post("/add")
def add_ticker(exchange: str = Form(...), ticker: str = Form(...)):
    """Handle the form submission to add a new ticker to the tracking list."""
    # Load current tickers from file
    try:
        with open("tickers.json", "r") as f:
            tickers = json.load(f)
    except FileNotFoundError:
        tickers = []
    # Avoid duplicates: check if this exchange/ticker combo is already tracked
    exists = any(t.get("exchange") == exchange and t.get("ticker") == ticker for t in tickers)
    if not exists:
        tickers.append({ "exchange": exchange, "ticker": ticker })
        tickers.sort(key=lambda x: (x["exchange"], x["ticker"]))
        # Save updated list back to JSON file
        with open("tickers.json", "w") as f:
            json.dump(tickers, f, indent=4)
        # If this exchange is new, initialize its status info
        if exchange not in status_info:
            status_info[exchange] = {
                "status": "Running",
                "last_updated": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            }
    # Redirect back to the main dashboard (POST/Redirect/GET pattern)
    return RedirectResponse(url="/", status_code=303)

@app.post("/delete")
def delete_ticker(exchange: str = Form(...), ticker: str = Form(...)):
    """Handle the deletion of a ticker from the tracking list."""
    # Load current tickers from file
    try:
        with open("tickers.json", "r") as f:
            tickers = json.load(f)
    except FileNotFoundError:
        tickers = []
    # Remove the specified ticker (if it exists in the list)
    tickers = [t for t in tickers if not (t.get("exchange") == exchange and t.get("ticker") == ticker)]
    tickers.sort(key=lambda x: (x["exchange"], x["ticker"]))
    # Save the pruned list back to JSON
    with open("tickers.json", "w") as f:
        json.dump(tickers, f, indent=4)
    # If the exchange has no more tickers, remove it from status tracking
    remaining_exchanges = { t["exchange"] for t in tickers }
    if exchange not in remaining_exchanges:
        status_info.pop(exchange, None)
    return RedirectResponse(url="/", status_code=303)


In [11]:
# 에러 핸들러 등록
@app.exception_handler(Exception)
async def global_exception_handler(request: Request, exc: Exception):
    log.error(f"❗ Error occurred: {exc}")
    # 예외 발생 시 자동으로 로그 남기고, 사용자에게는 500 응답
    return JSONResponse(status_code=500, content={"detail": "Internal Server Error"})

In [12]:
pip install nest_asyncio


[notice] A new release of pip available: 22.1.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [13]:
pip install uvicorn


[notice] A new release of pip available: 22.1.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [14]:
# jupyter 실행 시 필요
import nest_asyncio
import uvicorn

In [15]:

if __name__ == "__main__":
    nest_asyncio.apply()
    uvicorn.run(app)

INFO:     Started server process [20660]
INFO:     Waiting for application startup.
ERROR:    Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\starlette\routing.py", line 692, in lifespan
    async with self.lifespan_context(app) as maybe_state:
  File "C:\ProgramData\Anaconda3\lib\contextlib.py", line 181, in __aenter__
    return await self.gen.__anext__()
  File "C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_20660/691386356.py", line 21, in lifespan
    if ex not in status_info:
NameError: name 'status_info' is not defined

ERROR:    Application startup failed. Exiting.


SystemExit: 3

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3452: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
